In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

### KNN Regresor con los datos de Fernando

CSV completo 

In [2]:
df_original_fer = pd.read_csv('../data/datos_procesados.csv')
df_original_fer.head()

,Año,Nº puertas,Potencia,Velocidad máxima,Aceleracion,Consumo,Emisiones,Largo,Ancho,Alto,...,Carroceria_Monovolumen,Carroceria_Pick Up,Carroceria_Todoterreno,Carroceria_Turismo,Carroceria_Turismo familiar,Carroceria_Vehículo comercial,Etiqueta_C,Etiqueta_Cero,Etiqueta_ECO,Etiqueta_Sin etiqueta
0,2019.0,5.0,150.0,212.0,8.7,4.5,117.0,4548.0,1816.0,1439.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2009.0,3.0,110.0,190.0,10.7,4.5,119.0,4199.0,1779.0,1469.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2007.0,5.0,177.0,200.0,9.3,7.0,185.0,4395.0,1815.0,1720.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011.0,5.0,143.0,190.0,11.4,6.5,172.0,4629.0,1880.0,1653.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013.0,5.0,313.0,250.0,5.3,6.4,169.0,4969.0,1911.0,1420.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df_original_fer.columns

Index(['Año', 'Nº puertas', 'Potencia', 'Velocidad máxima', 'Aceleracion',
       'Consumo', 'Emisiones', 'Largo', 'Ancho', 'Alto', 'KM_log',
       'Precio_log', 'C_Eléctrico', 'C_Gasolina', 'C_Híbrido', 'C_Otros',
       'Ca_Manual', 'Ca_Otro', 'Color_Azul', 'Color_Beige', 'Color_Blanco',
       'Color_Gris', 'Color_Marron', 'Color_Naranja', 'Color_Negro',
       'Color_Otro', 'Color_Rojo', 'Color_Verde', 'Color_Violeta',
       'Marca_encoded', 'Carroceria_Descapotable', 'Carroceria_Monovolumen',
       'Carroceria_Pick Up', 'Carroceria_Todoterreno', 'Carroceria_Turismo',
       'Carroceria_Turismo familiar', 'Carroceria_Vehículo comercial',
       'Etiqueta_C', 'Etiqueta_Cero', 'Etiqueta_ECO', 'Etiqueta_Sin etiqueta'],
      dtype='object')

In [4]:
df_fer_completo = df_original_fer.copy()

In [5]:
X = df_fer_completo.drop('Precio_log', axis=1)
y = df_fer_completo['Precio_log']

X.shape, y.shape

((52062, 40), (52062,))

In [6]:
# Normalizo 
scaler_x = MinMaxScaler()
scaler_x.fit(X)
X = scaler_x.transform(X)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

print(f"Conjunto de Train: {X_train.shape, y_train.shape}")
print(f"Conjunto de Test: {X_test.shape, y_test.shape}")

Conjunto de Train: ((41649, 40), (41649,))
Conjunto de Test: ((10413, 40), (10413,))


In [8]:
knn = KNeighborsRegressor(n_neighbors=3)

knn.fit(X_train, y_train)

yhat = knn.predict(X_test)

In [9]:
# Creo DF con resultados predichos y reales y diferencia porcedntual entre ellos
df_resultados = pd.DataFrame({'valor_real': y_test, 'valor_predicho': yhat})
df_resultados['diferencia'] = ((df_resultados['valor_real'] - df_resultados['valor_predicho'])/df_resultados['valor_real'])*100
promedio = df_resultados['diferencia'].mean()
print(f'Diferencia promedio entre valores reales y predichos: {promedio}')
print('_______')
print(df_resultados)

Diferencia promedio entre valores reales y predichos: 0.01289389711334575
_______
       valor_real  valor_predicho  diferencia
43661   10.126271       10.377920   -2.485110
6524     9.679781        9.695805   -0.165540
23482    9.729134        9.633203    0.986015
38964   11.000448       10.784339    1.964551
48831    9.209440        9.747887   -5.846686
...           ...             ...         ...
50116   10.595409       10.487655    1.016990
28595    9.814656        9.942252   -1.300057
35642   10.462846       10.232442    2.202122
44659    9.887155        9.992680   -1.067300
41797   10.036750       10.106292   -0.692875

[10413 rows x 3 columns]


In [10]:
RAE = np.sum(np.abs(np.subtract(y_test, yhat))) / np.sum(np.abs(np.subtract(y_test, np.mean(y_test))))
RSE = np.sum(np.square(np.subtract(y_test, yhat))) / np.sum(np.square(np.subtract(y_test, np.mean(y_test))))
r2_ajustada = 1 - (1 - knn.score(X_test, y_test))*(len(y_test) - 1)/(len(y_test) - X_test.shape[1] - 1)


print(f"MAE:\t {mean_absolute_error(y_test, yhat)}")
print(f"MSE:\t {mean_squared_error(y_test, yhat)}")
print(f"RMSE:\t {np.sqrt(mean_squared_error(y_test, yhat))}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"R**2:\t {r2_score(y_test, yhat)}")
print(f"Adjusted R**2:\t {r2_ajustada}")

MAE:	 0.1321459629983547
MSE:	 0.044224611549547335
RMSE:	 0.2102964848720666
RAE:	 0.33541471470177153
RSE:	 0.1549516394791501
R**2:	 0.8450483605208499
Adjusted R**2:	 0.844450783816341


In [11]:
metricas = {
    'MAE': mean_absolute_error(y_test, yhat),
    'MSE': mean_squared_error(y_test, yhat),
    'RMAE': np.sqrt(mean_squared_error(y_test, yhat)),
    'RAE': RAE,
    'RSE': RSE,
    'R**2': r2_score(y_test, yhat),
    'Adjusted R**2': r2_ajustada
}

df_metricas_fer_completo = pd.DataFrame(metricas, index=[0])
df_metricas_fer_completo

,MAE,MSE,RMAE,RAE,RSE,R**2,Adjusted R**2
0,0.132146,0.044225,0.210296,0.335415,0.154952,0.845048,0.844451


Datos de Feature selection

In [12]:
df_original_fer.columns

Index(['Año', 'Nº puertas', 'Potencia', 'Velocidad máxima', 'Aceleracion',
       'Consumo', 'Emisiones', 'Largo', 'Ancho', 'Alto', 'KM_log',
       'Precio_log', 'C_Eléctrico', 'C_Gasolina', 'C_Híbrido', 'C_Otros',
       'Ca_Manual', 'Ca_Otro', 'Color_Azul', 'Color_Beige', 'Color_Blanco',
       'Color_Gris', 'Color_Marron', 'Color_Naranja', 'Color_Negro',
       'Color_Otro', 'Color_Rojo', 'Color_Verde', 'Color_Violeta',
       'Marca_encoded', 'Carroceria_Descapotable', 'Carroceria_Monovolumen',
       'Carroceria_Pick Up', 'Carroceria_Todoterreno', 'Carroceria_Turismo',
       'Carroceria_Turismo familiar', 'Carroceria_Vehículo comercial',
       'Etiqueta_C', 'Etiqueta_Cero', 'Etiqueta_ECO', 'Etiqueta_Sin etiqueta'],
      dtype='object')

In [13]:
df_fer_feature_selection =df_original_fer.copy()

In [14]:
df_fer_feature_selection = df_fer_feature_selection.drop(['C_Eléctrico', 'C_Gasolina', 'C_Híbrido', 'C_Otros',
       'Ca_Manual', 'Ca_Otro', 'Color_Azul', 'Color_Beige', 'Color_Blanco',
       'Color_Gris', 'Color_Marron', 'Color_Naranja', 'Color_Negro',
       'Color_Otro', 'Color_Rojo', 'Color_Verde', 'Color_Violeta','Etiqueta_C', 'Etiqueta_Cero', 'Etiqueta_ECO', 'Etiqueta_Sin etiqueta'], axis=1)

In [15]:
df_fer_feature_selection

,Año,Nº puertas,Potencia,Velocidad máxima,Aceleracion,Consumo,Emisiones,Largo,Ancho,Alto,KM_log,Precio_log,Marca_encoded,Carroceria_Descapotable,Carroceria_Monovolumen,Carroceria_Pick Up,Carroceria_Todoterreno,Carroceria_Turismo,Carroceria_Turismo familiar,Carroceria_Vehículo comercial
0,2019.0,5.0,150.0,212.0,8.7,4.5,117.0,4548.0,1816.0,1439.0,11.289744,9.740439,36.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2009.0,3.0,110.0,190.0,10.7,4.5,119.0,4199.0,1779.0,1469.0,12.166817,9.082621,44.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2007.0,5.0,177.0,200.0,9.3,7.0,185.0,4395.0,1815.0,1720.0,12.262836,9.200391,43.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2011.0,5.0,143.0,190.0,11.4,6.5,172.0,4629.0,1880.0,1653.0,12.168111,9.735128,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2013.0,5.0,313.0,250.0,5.3,6.4,169.0,4969.0,1911.0,1420.0,12.280025,10.184938,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52057,2010.0,3.0,78.0,165.0,12.3,5.9,138.0,4063.0,1721.0,1446.0,11.477319,8.947806,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
52058,2019.0,5.0,150.0,200.0,9.4,5.4,139.0,4486.0,1839.0,1673.0,11.744045,10.162037,44.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
52059,2020.0,5.0,95.0,185.0,11.2,3.7,97.0,4053.0,1751.0,1461.0,11.759793,9.539716,44.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
52060,2019.0,5.0,125.0,200.0,10.0,4.9,107.0,4378.0,1825.0,1471.0,10.682583,9.846970,15.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
X = df_fer_feature_selection.drop('Precio_log', axis=1)
y = df_fer_feature_selection['Precio_log']

X.shape, y.shape

((52062, 19), (52062,))

In [17]:
# Normalizo 
scaler_x = MinMaxScaler()
scaler_x.fit(X)
X = scaler_x.transform(X)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

print(f"Conjunto de Train: {X_train.shape, y_train.shape}")
print(f"Conjunto de Test: {X_test.shape, y_test.shape}")

Conjunto de Train: ((41649, 19), (41649,))
Conjunto de Test: ((10413, 19), (10413,))


In [19]:
knn = KNeighborsRegressor(n_neighbors=3)

knn.fit(X_train, y_train)

yhat = knn.predict(X_test)

In [20]:
# Creo DF con resultados predichos y reales y diferencia porcedntual entre ellos
df_resultados = pd.DataFrame({'valor_real': y_test, 'valor_predicho': yhat})
df_resultados['diferencia'] = ((df_resultados['valor_real'] - df_resultados['valor_predicho'])/df_resultados['valor_real'])*100
promedio = df_resultados['diferencia'].mean()
print(f'Diferencia promedio entre valores reales y predichos: {promedio}')
print('_______')
print(df_resultados)

Diferencia promedio entre valores reales y predichos: 0.011615452210794705
_______
       valor_real  valor_predicho  diferencia
43661   10.126271       10.258311   -1.303936
6524     9.679781        9.617661    0.641750
23482    9.729134        9.768608   -0.405728
38964   11.000448       10.975503    0.226771
48831    9.209440        9.426298   -2.354739
...           ...             ...         ...
50116   10.595409       10.487655    1.016990
28595    9.814656        9.634706    1.833483
35642   10.462846       10.490266   -0.262065
44659    9.887155        9.894112   -0.070369
41797   10.036750       10.144169   -1.070259

[10413 rows x 3 columns]


In [21]:
RAE = np.sum(np.abs(np.subtract(y_test, yhat))) / np.sum(np.abs(np.subtract(y_test, np.mean(y_test))))
RSE = np.sum(np.square(np.subtract(y_test, yhat))) / np.sum(np.square(np.subtract(y_test, np.mean(y_test))))
r2_ajustada = 1 - (1 - knn.score(X_test, y_test))*(len(y_test) - 1)/(len(y_test) - X_test.shape[1] - 1)


print(f"MAE:\t {mean_absolute_error(y_test, yhat)}")
print(f"MSE:\t {mean_squared_error(y_test, yhat)}")
print(f"RMSE:\t {np.sqrt(mean_squared_error(y_test, yhat))}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"R**2:\t {r2_score(y_test, yhat)}")
print(f"Adjusted R**2:\t {r2_ajustada}")

MAE:	 0.10326573069873952
MSE:	 0.025166695040102598
RMSE:	 0.15864014321760617
RAE:	 0.26211050882590287
RSE:	 0.08817761242214027
R**2:	 0.9118223875778597
Adjusted R**2:	 0.9116611853613659


In [22]:
metricas = {
    'MAE': mean_absolute_error(y_test, yhat),
    'MSE': mean_squared_error(y_test, yhat),
    'RMAE': np.sqrt(mean_squared_error(y_test, yhat)),
    'RAE': RAE,
    'RSE': RSE,
    'R**2': r2_score(y_test, yhat),
    'Adjusted R**2': r2_ajustada
}

df_metricas_fer_feature_selection = pd.DataFrame(metricas, index=[0])
df_metricas_fer_feature_selection



,MAE,MSE,RMAE,RAE,RSE,R**2,Adjusted R**2
0,0.103266,0.025167,0.15864,0.262111,0.088178,0.911822,0.911661


### KNN Regresor con los datos de Kike

CSV completo

In [23]:
df_original_kike = pd.read_csv('../data/datos_procesadoslog.csv')
df_original_kike.head()

,KM,Año,Combustible,Ciudad,Nº puertas,Cambio,Color,Marca,Modelo,Potencia,Tipo de carrocería,Velocidad máxima,Aceleracion,Consumo,Emisiones,Dimensiones,Etiqueta medioambiental,Precio,Largo
0,11.289744,7.610853,0.0,5.683580,1.791759,0.000000,0.693147,3.610918,5.843544,5.017280,2.079442,5.361292,2.272126,1.704748,4.770685,7.149917,0.693147,9.740439,NaN
1,12.166817,7.605890,0.0,4.543295,1.386294,0.693147,1.098612,3.806662,5.624018,4.709530,1.945910,5.252273,2.459589,1.704748,4.787492,6.133398,0.000000,9.082621,NaN
2,12.262836,7.604894,0.0,4.543295,1.791759,0.693147,1.609438,3.784190,6.091310,5.181784,1.791759,5.303305,2.332144,2.079442,5.225747,6.813445,0.000000,9.200391,NaN
3,12.168111,7.606885,0.0,4.543295,1.791759,0.693147,1.609438,1.386294,6.061457,4.969813,1.791759,5.252273,2.517696,2.014903,5.153292,7.268223,0.000000,9.735128,NaN
4,12.280025,7.607878,0.0,4.543295,1.791759,1.386294,1.609438,1.386294,4.174387,5.749393,1.945910,5.525453,1.840550,2.001480,5.135798,7.689371,0.000000,10.184938,NaN


In [24]:
df_kike_completo = df_original_kike.copy()

In [25]:
df_kike_completo = df_kike_completo.drop(['Largo'], axis=1)

In [26]:
X = df_kike_completo.drop('Precio', axis=1)
y = df_kike_completo['Precio']

X.shape, y.shape

((60770, 17), (60770,))

In [27]:
# Normalizo 
scaler_x = MinMaxScaler()
scaler_x.fit(X)
X = scaler_x.transform(X)

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

print(f"Conjunto de Train: {X_train.shape, y_train.shape}")
print(f"Conjunto de Test: {X_test.shape, y_test.shape}")

Conjunto de Train: ((48616, 17), (48616,))
Conjunto de Test: ((12154, 17), (12154,))


In [29]:
knn = KNeighborsRegressor(n_neighbors=3)

knn.fit(X_train, y_train)

yhat = knn.predict(X_test)

In [30]:
# Creo DF con resultados predichos y reales y diferencia porcedntual entre ellos
df_resultados = pd.DataFrame({'valor_real': y_test, 'valor_predicho': yhat})
df_resultados['diferencia'] = ((df_resultados['valor_real'] - df_resultados['valor_predicho'])/df_resultados['valor_real'])*100
promedio = df_resultados['diferencia'].mean()
print(f'Diferencia promedio entre valores reales y predichos: {promedio}')
print('_______')
print(df_resultados)

Diferencia promedio entre valores reales y predichos: -0.08966512741500236
_______
       valor_real  valor_predicho  diferencia
5599     9.102867        9.153617   -0.557523
52407    9.092795        9.428797   -3.695260
23715    8.962007        9.431404   -5.237626
26626   10.126271       10.350369   -2.213032
49647    9.770013        9.728381    0.426124
...           ...             ...         ...
34506   10.315299       10.290079    0.244494
1660     9.998389       10.013377   -0.149904
52172    9.349319        9.383770   -0.368482
47624    9.332204        9.397719   -0.702034
53464    9.472397        9.467005    0.056926

[12154 rows x 3 columns]


In [31]:
RAE = np.sum(np.abs(np.subtract(y_test, yhat))) / np.sum(np.abs(np.subtract(y_test, np.mean(y_test))))
RSE = np.sum(np.square(np.subtract(y_test, yhat))) / np.sum(np.square(np.subtract(y_test, np.mean(y_test))))
r2_ajustada = 1 - (1 - knn.score(X_test, y_test))*(len(y_test) - 1)/(len(y_test) - X_test.shape[1] - 1)


print(f"MAE:\t {mean_absolute_error(y_test, yhat)}")
print(f"MSE:\t {mean_squared_error(y_test, yhat)}")
print(f"RMSE:\t {np.sqrt(mean_squared_error(y_test, yhat))}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"R**2:\t {r2_score(y_test, yhat)}")
print(f"Adjusted R**2:\t {r2_ajustada}")

MAE:	 0.14698042746389492
MSE:	 0.05825128554178374
RMSE:	 0.2413530309355649
RAE:	 0.3787483362820017
RSE:	 0.21179890572005208
R**2:	 0.7882010942799479
Adjusted R**2:	 0.7879044082716057


In [32]:
metrics = {
    'MAE': mean_absolute_error(y_test, yhat),
    'MSE': mean_squared_error(y_test, yhat),
    'RMAE': np.sqrt(mean_squared_error(y_test, yhat)),
    'RAE': RAE,
    'RSE': RSE,
    'R**2': r2_score(y_test, yhat),
    'Adjusted R**2': r2_ajustada
}

df_metricas_kike_completo = pd.DataFrame(metrics, index=[0])

df_metricas_kike_completo

,MAE,MSE,RMAE,RAE,RSE,R**2,Adjusted R**2
0,0.14698,0.058251,0.241353,0.378748,0.211799,0.788201,0.787904


Datos de Feature selection

In [33]:
df_original_kike.columns

Index(['KM', 'Año', 'Combustible', 'Ciudad', 'Nº puertas', 'Cambio', 'Color',
       'Marca', 'Modelo', 'Potencia', 'Tipo de carrocería', 'Velocidad máxima',
       'Aceleracion', 'Consumo', 'Emisiones', 'Dimensiones',
       'Etiqueta medioambiental', 'Precio', 'Largo'],
      dtype='object')

In [34]:
df_kike_feature_selection =df_original_kike.copy()

In [35]:
df_kike_feature_selection

,KM,Año,Combustible,Ciudad,Nº puertas,Cambio,Color,Marca,Modelo,Potencia,Tipo de carrocería,Velocidad máxima,Aceleracion,Consumo,Emisiones,Dimensiones,Etiqueta medioambiental,Precio,Largo
0,11.289744,7.610853,0.000000,5.683580,1.791759,0.000000,0.693147,3.610918,5.843544,5.017280,2.079442,5.361292,2.272126,1.704748,4.770685,7.149917,0.693147,9.740439,NaN
1,12.166817,7.605890,0.000000,4.543295,1.386294,0.693147,1.098612,3.806662,5.624018,4.709530,1.945910,5.252273,2.459589,1.704748,4.787492,6.133398,0.000000,9.082621,NaN
2,12.262836,7.604894,0.000000,4.543295,1.791759,0.693147,1.609438,3.784190,6.091310,5.181784,1.791759,5.303305,2.332144,2.079442,5.225747,6.813445,0.000000,9.200391,NaN
3,12.168111,7.606885,0.000000,4.543295,1.791759,0.693147,1.609438,1.386294,6.061457,4.969813,1.791759,5.252273,2.517696,2.014903,5.153292,7.268223,0.000000,9.735128,NaN
4,12.280025,7.607878,0.000000,4.543295,1.791759,1.386294,1.609438,1.386294,4.174387,5.749393,1.945910,5.525453,1.840550,2.001480,5.135798,7.689371,0.000000,10.184938,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60765,11.477319,7.606387,1.098612,5.241747,1.386294,0.693147,2.197225,0.693147,5.918894,4.369448,1.945910,5.111988,2.587764,1.931521,4.934474,5.786897,0.693147,8.947806,NaN
60766,11.744045,7.610853,0.000000,4.624973,1.791759,0.000000,1.609438,3.806662,6.272877,5.017280,1.791759,5.303305,2.341806,1.856298,4.941642,7.024649,0.693147,10.162037,NaN
60767,11.759793,7.611348,0.000000,4.624973,1.791759,0.693147,1.386294,3.806662,6.033086,4.564348,1.945910,5.225747,2.501436,1.547563,4.584967,5.746203,0.693147,9.539716,NaN
60768,10.682583,7.610853,1.098612,5.789960,1.791759,0.693147,2.197225,2.772589,5.521461,4.836282,1.945910,5.303305,2.397895,1.774952,4.682131,6.749931,0.693147,9.846970,NaN


In [36]:
df_kike_feature_selection = df_kike_feature_selection.drop(['Precio','Etiqueta medioambiental','Largo','Consumo','Emisiones','Ciudad', 'Nº puertas','Color', 'Combustible'], axis=1)

In [37]:
# Normalizo 
scaler_x = MinMaxScaler()
scaler_x.fit(X)
X = scaler_x.transform(X)

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

print(f"Conjunto de Train: {X_train.shape, y_train.shape}")
print(f"Conjunto de Test: {X_test.shape, y_test.shape}")

Conjunto de Train: ((48616, 17), (48616,))
Conjunto de Test: ((12154, 17), (12154,))


In [39]:
knn = KNeighborsRegressor(n_neighbors=3)

knn.fit(X_train, y_train)

yhat = knn.predict(X_test)

In [40]:
# Creo DF con resultados predichos y reales y diferencia porcedntual entre ellos
df_resultados = pd.DataFrame({'valor_real': y_test, 'valor_predicho': yhat})
df_resultados['diferencia'] = ((df_resultados['valor_real'] - df_resultados['valor_predicho'])/df_resultados['valor_real'])*100
promedio = df_resultados['diferencia'].mean()
print(f'Diferencia promedio entre valores reales y predichos: {promedio}')
print('_______')
print(df_resultados)

Diferencia promedio entre valores reales y predichos: -0.08966512741500236
_______
       valor_real  valor_predicho  diferencia
5599     9.102867        9.153617   -0.557523
52407    9.092795        9.428797   -3.695260
23715    8.962007        9.431404   -5.237626
26626   10.126271       10.350369   -2.213032
49647    9.770013        9.728381    0.426124
...           ...             ...         ...
34506   10.315299       10.290079    0.244494
1660     9.998389       10.013377   -0.149904
52172    9.349319        9.383770   -0.368482
47624    9.332204        9.397719   -0.702034
53464    9.472397        9.467005    0.056926

[12154 rows x 3 columns]


In [41]:
RAE = np.sum(np.abs(np.subtract(y_test, yhat))) / np.sum(np.abs(np.subtract(y_test, np.mean(y_test))))
RSE = np.sum(np.square(np.subtract(y_test, yhat))) / np.sum(np.square(np.subtract(y_test, np.mean(y_test))))
r2_ajustada = 1 - (1 - knn.score(X_test, y_test))*(len(y_test) - 1)/(len(y_test) - X_test.shape[1] - 1)


print(f"MAE:\t {mean_absolute_error(y_test, yhat)}")
print(f"MSE:\t {mean_squared_error(y_test, yhat)}")
print(f"RMSE:\t {np.sqrt(mean_squared_error(y_test, yhat))}")
print(f"RAE:\t {RAE}")
print(f"RSE:\t {RSE}")
print(f"R**2:\t {r2_score(y_test, yhat)}")
print(f"Adjusted R**2:\t {r2_ajustada}")

MAE:	 0.14698042746389492
MSE:	 0.05825128554178374
RMSE:	 0.2413530309355649
RAE:	 0.3787483362820017
RSE:	 0.21179890572005208
R**2:	 0.7882010942799479
Adjusted R**2:	 0.7879044082716057


In [42]:
metricas = {
    'MAE': mean_absolute_error(y_test, yhat),
    'MSE': mean_squared_error(y_test, yhat),
    'RMAE': np.sqrt(mean_squared_error(y_test, yhat)),
    'RAE': RAE,
    'RSE': RSE,
    'R**2': r2_score(y_test, yhat),
    'Adjusted R**2': r2_ajustada
}

df_metricas_kike_feature_selection = pd.DataFrame(metricas, index=[0])
df_metricas_kike_feature_selection

,MAE,MSE,RMAE,RAE,RSE,R**2,Adjusted R**2
0,0.14698,0.058251,0.241353,0.378748,0.211799,0.788201,0.787904


In [43]:
df_metricas_totales = pd.concat([df_metricas_fer_completo, df_metricas_fer_feature_selection, df_metricas_kike_completo, df_metricas_kike_feature_selection], ignore_index= True, axis=0)
df_metricas_totales['DF'] = (['df_metricas_fer_completo', 'df_metricas_fer_feature_selection', 'df_metricas_kike_completo', 'df_metricas_kike_feature_selection'])
df_metricas_totales

,MAE,MSE,RMAE,RAE,RSE,R**2,Adjusted R**2,DF
0,0.132146,0.044225,0.210296,0.335415,0.154952,0.845048,0.844451,df_metricas_fer_completo
1,0.103266,0.025167,0.158640,0.262111,0.088178,0.911822,0.911661,df_metricas_fer_feature_selection
2,0.146980,0.058251,0.241353,0.378748,0.211799,0.788201,0.787904,df_metricas_kike_completo
3,0.146980,0.058251,0.241353,0.378748,0.211799,0.788201,0.787904,df_metricas_kike_feature_selection
